# Exercise 6

## SVM & Regularization


For this homework we consider a set of observations on a number of red and white wine varieties involving their chemical properties and ranking by tasters. Wine industry shows a recent growth spurt as social drinking is on the rise. The price of wine depends on a rather abstract concept of wine appreciation by wine tasters, opinion among whom may have a high degree of variability. Pricing of wine depends on such a volatile factor to some extent. Another key factor in wine certification and quality assessment is physicochemical tests which are laboratory-based and takes into account factors like acidity, pH level, presence of sugar and other chemical properties. For the wine market, it would be of interest if human quality of tasting can be related to the chemical properties of wine so that certification and quality assessment and assurance process is more controlled.

Two datasets are available of which one dataset is on red wine and have 1599 different varieties and the other is on white wine and have 4898 varieties. All wines are produced in a particular area of Portugal. Data are collected on 12 different properties of the wines one of which is Quality, based on sensory data, and the rest are on chemical properties of the wines including density, acidity, alcohol content etc. All chemical properties of wines are continuous variables. Quality is an ordinal variable with possible ranking from 1 (worst) to 10 (best). Each variety of wine is tasted by three independent tasters and the final rank assigned is the median rank given by the tasters.

A predictive model developed on this data is expected to provide guidance to vineyards regarding quality and price expected on their produce without heavy reliance on volatility of wine tasters.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_r = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_red.csv')
data_w = pd.read_csv('https://github.com/albahnsen/PracticalMachineLearningClass/raw/master/datasets/Wine_data_white.csv')

In [3]:
data = data_w.assign(type = 'white')

data = data.append(data_r.assign(type = 'red'), ignore_index=True)
data.sample(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type
1092,7.1,0.21,0.40,1.2,0.069,24.0,156.0,0.99280,3.42,0.43,10.6,6,white
3543,6.0,0.33,0.20,1.8,0.031,49.0,159.0,0.99190,3.41,0.53,11.0,6,white
4010,7.2,0.21,0.29,3.1,0.044,39.0,122.0,0.99143,3.00,0.60,11.3,6,white
4822,6.8,0.23,0.48,1.5,0.036,35.0,165.0,0.99162,3.18,0.45,11.3,6,white
3338,7.2,0.27,0.28,15.2,0.046,6.0,41.0,0.99665,3.17,0.39,10.9,6,white


# Exercise 6.1

Show the frecuency table of the quality by type of wine

In [4]:
frecuencia=data.pivot_table(values='pH', index=['type'],columns='quality', aggfunc='count')
frecuencia

quality,3,4,5,6,7,8,9
type,,,,,,,
red,10.0,53.0,681.0,638.0,199.0,18.0,NaN
white,20.0,163.0,1457.0,2198.0,880.0,175.0,5.0


# SVM

# Exercise 6.2

* Standarized the features (not the quality)
* Create a binary target for each type of wine
* Create two Linear SVM's for the white and red wines, repectively.


In [5]:
data.quality.value_counts()

6    2836
5    2138
7    1079
4     216
8     193
3      30
9       5
Name: quality, dtype: int64

In [6]:
data["quality_bin"]=np.where(data["quality"] <= 6, True, False)   

In [7]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,type,quality_bin
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6,white,True
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6,white,True
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6,white,True
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white,True
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6,white,True


In [8]:
data.quality_bin.value_counts()

True     5220
False    1277
Name: quality_bin, dtype: int64

### Separar por tipo de vino, y escoger las variables explicativas y la variable a dependiente:

In [9]:
data_red_wine=data[data['type']=='red']
X_red=data_red_wine[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']]
y_red=data_red_wine['quality_bin']

data_white_wine=data[data['type']=='white']
X_white=data_white_wine[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']]
y_white=data_white_wine['quality_bin']

## Train y test

In [10]:
from sklearn.model_selection import train_test_split  

#Train y test
X_train_w, X_test_w, y_train_w, y_test_w = train_test_split(X_white, y_white, test_size = 0.20)  
X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_red, y_red, test_size = 0.20)  

## Scaling

In [11]:
from sklearn import preprocessing
X_train_w = preprocessing.scale(X_train_w)
X_train_r = preprocessing.scale(X_train_r)
X_test_w = preprocessing.scale(X_test_w)
X_test_r = preprocessing.scale(X_test_r)


### SVM Red Wine

In [12]:
from sklearn.svm import SVC # "Support Vector Classifier"
clf = SVC(kernel='linear')
clf.fit(X_train_r, y_train_r)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

#### Evaluando el agloritmo

In [13]:
from sklearn.metrics import classification_report, confusion_matrix  
y_pred_r = clf.predict(X_test_r)  
print(confusion_matrix(y_test_r,y_pred_r))  
print(classification_report(y_test_r,y_pred_r))   

[[  0  34]
 [  0 286]]
             precision    recall  f1-score   support

      False       0.00      0.00      0.00        34
       True       0.89      1.00      0.94       286

avg / total       0.80      0.89      0.84       320



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### Normalizado

No cambian los resultados al normalizar o estandarizar, sin embarog, al hacerlo, converge mas rapido

### SVM White  Wine

In [14]:
clf = SVC(kernel='linear')
clf.fit(X_train_w, y_train_w)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

#### Evaluando el agloritmo

In [15]:
y_pred_w = clf.predict(X_test_w)  
print(confusion_matrix(y_test_w,y_pred_w))  
print(classification_report(y_test_w,y_pred_w))   

[[  0 210]
 [  0 770]]
             precision    recall  f1-score   support

      False       0.00      0.00      0.00       210
       True       0.79      1.00      0.88       770

avg / total       0.62      0.79      0.69       980



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# Exercise 6.3

Test the two SVM's using the different kernels (‘poly’, ‘rbf’, ‘sigmoid’)


### Red Wine

In [16]:
## Poly
clf = SVC(kernel='poly',degree=7)
clf.fit(X_train_r, y_train_r)
y_pred_r = clf.predict(X_test_r)  
print("Kernel: Poly\n")
print(confusion_matrix(y_test_r,y_pred_r))  
print(classification_report(y_test_r,y_pred_r))  

## rbf
clf = SVC(kernel='rbf')
clf.fit(X_train_r, y_train_r)
y_pred_r = clf.predict(X_test_r)  
print("Kernel: rbf\n")
print(confusion_matrix(y_test_r,y_pred_r))  
print(classification_report(y_test_r,y_pred_r))  

## sigmoid
clf = SVC(kernel='sigmoid')
clf.fit(X_train_r, y_train_r)
y_pred_r = clf.predict(X_test_r)  
print(" Kernel : Sigmoid\n")
print(confusion_matrix(y_test_r,y_pred_r))  
print(classification_report(y_test_r,y_pred_r))  

Kernel: Poly

[[  8  26]
 [ 10 276]]
             precision    recall  f1-score   support

      False       0.44      0.24      0.31        34
       True       0.91      0.97      0.94       286

avg / total       0.86      0.89      0.87       320

Kernel: rbf

[[ 12  22]
 [  6 280]]
             precision    recall  f1-score   support

      False       0.67      0.35      0.46        34
       True       0.93      0.98      0.95       286

avg / total       0.90      0.91      0.90       320

 Kernel : Sigmoid

[[  8  26]
 [ 30 256]]
             precision    recall  f1-score   support

      False       0.21      0.24      0.22        34
       True       0.91      0.90      0.90       286

avg / total       0.83      0.82      0.83       320



### White wine 

In [17]:
## Poly
clf = SVC(kernel='poly',degree=7)
clf.fit(X_train_w, y_train_w)
y_pred_w = clf.predict(X_test_w)  
print("Kernel: Poly\n")
print(confusion_matrix(y_test_w,y_pred_w))  
print(classification_report(y_test_w,y_pred_w))  

## rbf
clf = SVC(kernel='rbf')
clf.fit(X_train_w, y_train_w)
y_pred_w = clf.predict(X_test_w)  
print("Kernel: rbf\n")
print(confusion_matrix(y_test_w,y_pred_w))  
print(classification_report(y_test_w,y_pred_w))  

## sigmoid
clf = SVC(kernel='sigmoid')
clf.fit(X_train_w, y_train_w)
y_pred_w = clf.predict(X_test_w)  
print(" Kernel : Sigmoid\n")
print(confusion_matrix(y_test_w,y_pred_w))  
print(classification_report(y_test_w,y_pred_w))  

Kernel: Poly

[[ 48 162]
 [ 23 747]]
             precision    recall  f1-score   support

      False       0.68      0.23      0.34       210
       True       0.82      0.97      0.89       770

avg / total       0.79      0.81      0.77       980

Kernel: rbf

[[ 72 138]
 [ 30 740]]
             precision    recall  f1-score   support

      False       0.71      0.34      0.46       210
       True       0.84      0.96      0.90       770

avg / total       0.81      0.83      0.80       980

 Kernel : Sigmoid

[[ 88 122]
 [118 652]]
             precision    recall  f1-score   support

      False       0.43      0.42      0.42       210
       True       0.84      0.85      0.84       770

avg / total       0.75      0.76      0.75       980



# Exercise 6.4
Using the best SVM find the parameters that gives the best performance

'C': [0.1, 1, 10, 100, 1000], 'gamma': [0.01, 0.001, 0.0001]

In [18]:
C=[ 0.1, 1, 10, 100, 1000]
gamma=[0.01, 0.001, 0.0001]
from sklearn.metrics import accuracy_score

### Red Wine

In [19]:
for c in C:
    for g in gamma:
        ## Poly
        clf = SVC(kernel='poly',degree=7,C=c,gamma=g)
        clf.fit(X_train_r, y_train_r)
        y_pred_r = clf.predict(X_test_r)  
        print("Parametro C: ",c)
        print("Parametro gamma: ",g)
        print("Accuracy: ",accuracy_score(y_test_r,y_pred_r))  
        #print(confusion_matrix(y_test_w,y_pred_w))  
        print(classification_report(y_test_r,y_pred_r))  


Parametro C:  0.1
Parametro gamma:  0.01
Accuracy:  0.89375
             precision    recall  f1-score   support

      False       0.00      0.00      0.00        34
       True       0.89      1.00      0.94       286

avg / total       0.80      0.89      0.84       320

Parametro C:  0.1
Parametro gamma:  0.001
Accuracy:  0.89375
             precision    recall  f1-score   support

      False       0.00      0.00      0.00        34
       True       0.89      1.00      0.94       286

avg / total       0.80      0.89      0.84       320

Parametro C:  0.1
Parametro gamma:  0.0001
Accuracy:  0.89375
             precision    recall  f1-score   support

      False       0.00      0.00      0.00        34
       True       0.89      1.00      0.94       286

avg / total       0.80      0.89      0.84       320

Parametro C:  1
Parametro gamma:  0.01
Accuracy:  0.89375
             precision    recall  f1-score   support

      False       0.00      0.00      0.00        34
       

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Parametro C:  10
Parametro gamma:  0.01

### White Wine

In [20]:
for c in C:
    for g in gamma:
        ## Poly
        clf = SVC(kernel='poly',degree=7,C=c,gamma=g)
        clf.fit(X_train_w, y_train_w)
        y_pred_w = clf.predict(X_test_w)  
        print("Parametro C: ",c)
        print("Parametro gamma: ",g)
        print("Accuracy: ",accuracy_score(y_test_w,y_pred_w))  
        #print(confusion_matrix(y_test_w,y_pred_w))  
        print(classification_report(y_test_w,y_pred_w))  


Parametro C:  0.1
Parametro gamma:  0.01
Accuracy:  0.7857142857142857
             precision    recall  f1-score   support

      False       0.00      0.00      0.00       210
       True       0.79      1.00      0.88       770

avg / total       0.62      0.79      0.69       980

Parametro C:  0.1
Parametro gamma:  0.001
Accuracy:  0.7857142857142857
             precision    recall  f1-score   support

      False       0.00      0.00      0.00       210
       True       0.79      1.00      0.88       770

avg / total       0.62      0.79      0.69       980



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Parametro C:  0.1
Parametro gamma:  0.0001
Accuracy:  0.7857142857142857
             precision    recall  f1-score   support

      False       0.00      0.00      0.00       210
       True       0.79      1.00      0.88       770

avg / total       0.62      0.79      0.69       980

Parametro C:  1
Parametro gamma:  0.01
Accuracy:  0.7857142857142857
             precision    recall  f1-score   support

      False       0.00      0.00      0.00       210
       True       0.79      1.00      0.88       770

avg / total       0.62      0.79      0.69       980

Parametro C:  1
Parametro gamma:  0.001
Accuracy:  0.7857142857142857
             precision    recall  f1-score   support

      False       0.00      0.00      0.00       210
       True       0.79      1.00      0.88       770

avg / total       0.62      0.79      0.69       980

Parametro C:  1
Parametro gamma:  0.0001
Accuracy:  0.7857142857142857
             precision    recall  f1-score   support

      False       

Parametro C:  1000
Parametro gamma:  0.001c

# Exercise 6.5

Compare the results with other methods

### Red Wine

In [21]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='liblinear',C=1e9)


clf = LogisticRegression()
clf.fit(X_train_r, y_train_r)
y_pred_r = clf.predict(X_test_r)  
print("Accuracy: ",accuracy_score(y_test_r,y_pred_r))  
print("Confusion Matrix: \n ",confusion_matrix(y_test_r,y_pred_r))  
print(classification_report(y_test_r,y_pred_r)) 

Accuracy:  0.884375
Confusion Matrix: 
  [[ 12  22]
 [ 15 271]]
             precision    recall  f1-score   support

      False       0.44      0.35      0.39        34
       True       0.92      0.95      0.94       286

avg / total       0.87      0.88      0.88       320



### White Wine

In [22]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(solver='liblinear',C=1e9)

clf = LogisticRegression()
clf.fit(X_train_w, y_train_w)
y_pred_r = clf.predict(X_test_w)  
print("Accuracy: ",accuracy_score(y_test_w,y_pred_w))  
print("Confusion Matrix: \n ",confusion_matrix(y_test_w,y_pred_w))  
print(classification_report(y_test_w,y_pred_w)) 

Accuracy:  0.7857142857142857
Confusion Matrix: 
  [[  0 210]
 [  0 770]]
             precision    recall  f1-score   support

      False       0.00      0.00      0.00       210
       True       0.79      1.00      0.88       770

avg / total       0.62      0.79      0.69       980



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Se obtienen resultados similares a los obtenidos con el SVM

# Regularization

# Exercise 6.6


* Train a linear regression to predict wine quality (Continous)

* Analyze the coefficients

* Evaluate the RMSE

In [23]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
X=data[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']]
y=data['quality']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)  
clf = LinearRegression(normalize=True)
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)
print('RMSE:',np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

RMSE: 0.7073692780085403


In [24]:
list(zip(clf.coef_, X.columns))

[(0.061395872972993794, 'fixed acidity'),
 (-1.3198211885306308, 'volatile acidity'),
 (-0.08330206763111259, 'citric acid'),
 (0.04280396636470111, 'residual sugar'),
 (-0.5476638477843754, 'chlorides'),
 (0.005357935454639257, 'free sulfur dioxide'),
 (-0.002375944486074309, 'total sulfur dioxide'),
 (-51.53879060214959, 'density'),
 (0.423870216152956, 'pH'),
 (0.7421655859476448, 'sulphates'),
 (0.2668871452180932, 'alcohol')]

Los coeicientes son variables en cuanto a tamaño

# Exercise 6.7

* Estimate a ridge regression with alpha equals 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

### Alpha=0.1

In [25]:
# alpha=0 is equivalent to linear regression
from sklearn.linear_model import Ridge
ridgereg = Ridge(alpha=0.1, normalize=True)
ridgereg.fit(X_train, y_train)
y_pred = ridgereg.predict(X_test)
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

0.7123732789964877


In [26]:
list(zip(ridgereg.coef_, X.columns))

[(0.02502274663419073, 'fixed acidity'),
 (-1.1544714321271625, 'volatile acidity'),
 (0.04001876539963061, 'citric acid'),
 (0.026642287227670494, 'residual sugar'),
 (-1.0336818568861068, 'chlorides'),
 (0.004100646818293349, 'free sulfur dioxide'),
 (-0.0017574968642497754, 'total sulfur dioxide'),
 (-29.82470702946888, 'density'),
 (0.25059226151889147, 'pH'),
 (0.6349688054159802, 'sulphates'),
 (0.2564645631282619, 'alcohol')]

### Alpha=1

In [27]:
# alpha=0 is equivalent to linear regression
from sklearn.linear_model import Ridge
ridgereg = Ridge(alpha=1, normalize=True)
ridgereg.fit(X_train, y_train)
y_pred = ridgereg.predict(X_test)
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

0.7536207665179115


In [28]:
list(zip(ridgereg.coef_, X.columns))

[(-0.002758484490947164, 'fixed acidity'),
 (-0.5781607943837666, 'volatile acidity'),
 (0.17889790395568772, 'citric acid'),
 (0.005909564187723413, 'residual sugar'),
 (-1.3158822427130146, 'chlorides'),
 (0.001374135108243448, 'free sulfur dioxide'),
 (-0.0005725178808080742, 'total sulfur dioxide'),
 (-22.893829301955414, 'density'),
 (0.08625651715184338, 'pH'),
 (0.2926589425060024, 'sulphates'),
 (0.13651496066377228, 'alcohol')]

En la medida que aumentamos el alpha, la difenrencia en los valores de los coeficientes disminuye pero aumenta el error.

# Exercise 6.8

* Estimate a lasso regression with alpha equals 0.01, 0.1 and 1.
* Compare the coefficients with the linear regression
* Evaluate the RMSE

In [29]:
# try alpha=0.001 and examine coefficients
from sklearn.linear_model import Lasso
lassoreg = Lasso(alpha=0.01, normalize=True)
lassoreg.fit(X_train, y_train)
y_pred = lassoreg.predict(X_test)
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
list(zip(lassoreg.coef_, X.columns))

0.8593662004943828


[(-0.0, 'fixed acidity'),
 (-0.0, 'volatile acidity'),
 (0.0, 'citric acid'),
 (-0.0, 'residual sugar'),
 (-0.0, 'chlorides'),
 (0.0, 'free sulfur dioxide'),
 (-0.0, 'total sulfur dioxide'),
 (-0.0, 'density'),
 (0.0, 'pH'),
 (0.0, 'sulphates'),
 (0.0, 'alcohol')]

In [35]:
from sklearn.linear_model import Lasso
lassoreg = Lasso(alpha=0.1, normalize=True)
lassoreg.fit(X_train, y_train)
y_pred = lassoreg.predict(X_test)
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
list(zip(lassoreg.coef_, X.columns))

0.4064812266388907


[(0.0, 'fixed acidity'),
 (0.0, 'volatile acidity'),
 (-0.0, 'citric acid'),
 (0.0, 'residual sugar'),
 (0.0, 'chlorides'),
 (-0.0, 'free sulfur dioxide'),
 (0.0, 'total sulfur dioxide'),
 (0.0, 'density'),
 (-0.0, 'pH'),
 (-0.0, 'sulphates'),
 (-0.0, 'alcohol')]

In [37]:
from sklearn.linear_model import Lasso
lassoreg = Lasso(alpha=1, normalize=True)
lassoreg.fit(X_train, y_train)
y_pred = lassoreg.predict(X_test)
print(np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
list(zip(lassoreg.coef_, X.columns))

0.4064812266388907


[(0.0, 'fixed acidity'),
 (0.0, 'volatile acidity'),
 (-0.0, 'citric acid'),
 (0.0, 'residual sugar'),
 (0.0, 'chlorides'),
 (-0.0, 'free sulfur dioxide'),
 (0.0, 'total sulfur dioxide'),
 (0.0, 'density'),
 (-0.0, 'pH'),
 (-0.0, 'sulphates'),
 (-0.0, 'alcohol')]

En los 3 casos, los coeficientes son iguales a 0

# Exercise 6.9

* Create a binary target

* Train a logistic regression to predict wine quality (binary)

* Analyze the coefficients

* Evaluate the f1score

In [30]:

X=data[['fixed acidity','volatile acidity','citric acid','residual sugar','chlorides','free sulfur dioxide','total sulfur dioxide','density','pH','sulphates','alcohol']]
y=data['quality_bin']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)  


X_train= preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)

clf = LogisticRegression()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)  
print("Accuracy: ",accuracy_score(y_test,y_pred))  
print("Confusion Matrix: \n ",confusion_matrix(y_test,y_pred))  
print(classification_report(y_test,y_pred)) 



Accuracy:  0.816923076923077
Confusion Matrix: 
  [[ 70 201]
 [ 37 992]]
             precision    recall  f1-score   support

      False       0.65      0.26      0.37       271
       True       0.83      0.96      0.89      1029

avg / total       0.79      0.82      0.78      1300



In [31]:
list(zip(clf.coef_, data.columns))

[(array([-0.52747511,  0.54548481,  0.02687156, -0.79179878,  0.23975407,
         -0.24615168,  0.34831857,  0.78471   , -0.34848215, -0.36291321,
         -0.71476875]), 'fixed acidity')]

# Exercise 6.10

* Estimate a regularized logistic regression using:
* C = 0.01, 0.1 & 1.0
* penalty = ['l1, 'l2']
* Compare the coefficients and the f1score

In [32]:
C=[ 0.01, 0.1 ,1.0]
penalty=["l1", "l2"]

In [33]:
for c in C:
    for p in penalty:
        logreg = LogisticRegression(C=c, penalty=p,solver='liblinear')
        logreg.fit(X_train, y_train)
        y_pred = clf.predict(X_test)  
        print("Parametro C: ",c)
        print("Parametro gamma: ",p)
        print("Coeficientes: ",logreg.coef_)  
        print(classification_report(y_test,y_pred))  

Parametro C:  0.01
Parametro gamma:  l1
Coeficientes:  [[ 0.          0.29778715  0.          0.          0.          0.
   0.          0.          0.         -0.05101922 -0.77275996]]
             precision    recall  f1-score   support

      False       0.65      0.26      0.37       271
       True       0.83      0.96      0.89      1029

avg / total       0.79      0.82      0.78      1300

Parametro C:  0.01
Parametro gamma:  l2
Coeficientes:  [[-0.16636871  0.34392453 -0.03527468 -0.29466125  0.19952218 -0.15214483
   0.18302499  0.26170105 -0.13046513 -0.22659669 -0.67254725]]
             precision    recall  f1-score   support

      False       0.65      0.26      0.37       271
       True       0.83      0.96      0.89      1029

avg / total       0.79      0.82      0.78      1300

Parametro C:  0.1
Parametro gamma:  l1
Coeficientes:  [[-0.28188986  0.53984953  0.         -0.47172697  0.24639809 -0.19494195
   0.24481692  0.33595701 -0.20218641 -0.29499087 -0.87091634]]


Se obtienen resultados iguales en el f1-score, al cambiar los parametros.